In [86]:
import okx.MarketData as MarketData
import pandas as pd

flag = "0"  # 0:现货实时, 1:模拟盘
market = MarketData.MarketAPI(flag=flag)

# 获取ETH-USDT 15分钟K线，最近500条
result = market.get_candlesticks(
    instId="GRASS-USDT-SWAP",
    bar="1H",
    limit="100"  # 最大500条
)

data = result['data']  # 列表，每项：[ts, o, h, l, c, vol, volCcy, ...]
df = pd.DataFrame(data, columns=[
    'ts', 'open', 'high', 'low', 'close', 
    'vol', 'volCcy', 'volCcyQuote', 'confirm'
])

df['ts'] = pd.to_datetime(df['ts'].astype(int), unit='ms')
df = df.astype({'open':float, 'high':float, 'low':float, 'close':float, 'vol':float})
# df = df[::-1].reset_index(drop=True)  # 旧→新排序

df.head()


,ts,open,high,low,close,vol,volCcy,volCcyQuote,confirm
0,2025-11-23 08:00:00,0.3744,0.3795,0.3605,0.3616,3192883.0,3192883,1180054.108,0
1,2025-11-23 07:00:00,0.3762,0.3784,0.3704,0.3745,1955674.0,1955674,731037.812,1
2,2025-11-23 06:00:00,0.3837,0.3852,0.3740,0.3760,2172043.0,2172043,824782.5775,1
3,2025-11-23 05:00:00,0.3749,0.3873,0.3714,0.3837,1921225.0,1921225,724809.8904,1
4,2025-11-23 04:00:00,0.3887,0.3888,0.3748,0.3748,2807006.0,2807006,1067316.1168,1


In [87]:
import talib as ta
import numpy as np

periods = 10
multiplier = 3.0

[high, low, close] = [df["high"], df["low"], df["close"]]

hl2 = (high + low) / 2
atr = ta.ATR(high[::-1], low[::-1], close[::-1], periods)[::-1].reset_index(drop=True)
# atr = np.round(atr, 4)

up = hl2 - multiplier * atr
up1 = up[1:]

up1 = up1[:np.where(np.isnan(up1))[0][0]]
up1 = up1.reset_index(drop=True)
up = up[:len(up1)]

dn = hl2 + multiplier * atr
dn1 = dn[1:]

dn1 = up1[:np.where(np.isnan(dn1))[0][0]]
dn1 = dn1.reset_index(drop=True)
dn = dn[:len(dn1)]

close1 = close[1:][:len(up1)].reset_index(drop=True)

In [88]:
# 假设起始为下跌趋势
trend = [True] * (len(up) + 1)

up_max = up1[len(up1) - 1]
dn_min = dn1[len(dn1) - 1]

for i in range(len(up) - 1, 0, -1):
    curr, prev = i, i+1
    # 下跌趋势
    if trend[prev] == True:
        # 计算上轨最低点
        dn_min = dn_min if dn_min < dn[curr] else dn[curr]
        # 判断当前k收盘价是否 “上穿” 上轨最低点
        if close[curr] > dn_min:
            trend[curr] = False if close[curr] > dn_min else True
            up_max = up[curr]
    
    # 上升趋势
    else:
        # 计算下轨最高点
        up_max = up_max if up_max > up[curr] else up[curr]
        # 判断当前k收盘价是否 “下穿” 下轨最高点
        if close[curr] < up_max:
            trend[curr] = True if close[curr] < up_max else False
            dn_min = dn[curr]

trend

[True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True]

In [89]:
trend = [True] * len(up)
final_up = []
final_dn = []
up_max = up[len(up) - 1]
dn_min = dn[len(dn) - 1]

for i in range(len(up) - 1, 0, -1):
    up_max = max(up[i], up1[i]) if close1[i] > up1[i] else up_max
    final_up.append(up_max)

    dn_min = min(dn[i], dn1[i]) if close1[i] < dn1[i] else dn_min
    final_dn.append(dn_min)

    # if close > dn1[i]

final_up[::-1]

[np.float64(0.3283345332516764),
 np.float64(0.3283345332516764),
 np.float64(0.3279574484588597),
 np.float64(0.32834160939873297),
 np.float64(0.32834160939873297),
 np.float64(0.3262295659985922),
 np.float64(0.32307173999843575),
 np.float64(0.3217805432079454),
 np.float64(0.3217805432079454),
 np.float64(0.31986727023105044),
 np.float64(0.31432474470116717),
 np.float64(0.31601635148292245),
 np.float64(0.3189237238699138),
 np.float64(0.3189237238699138),
 np.float64(0.3129374709665709),
 np.float64(0.30256941218507877),
 np.float64(0.30164557059886277),
 np.float64(0.3038284117765142),
 np.float64(0.3038284117765142),
 np.float64(0.32316408861298046),
 np.float64(0.32316408861298046),
 np.float64(0.31266009845886716),
 np.float64(0.30194455384318575),
 np.float64(0.2853161709368731),
 np.float64(0.28411796770763675),
 np.float64(0.2796977418973741),
 np.float64(0.27598486653996807),
 np.float64(0.28186651837774235),
 np.float64(0.28186651837774235),
 np.float64(0.2852092819478